# Lecture 6 - Introduction to Data & Pandas
In this lecture, students will learn about how to work with data and use the most popular data analysis and manipulation library, `pandas`. This is one of a few lectures on `pandas` and related concepts. In this particular lecture, we cover the following topics among others.
- Basic Pandas functionality
- Pandas data types
- Key methods for Pandas data types
- Saving and loading data
- Data cleaning
- Reshaping
- Merging
- Grouping and Aggregating
- Time series data

This lecture follows QuantEcon's lecture on [Pandas](https://datascience.quantecon.org/pandas/index.html) more so than previous lectures and uses many of the same examples and code. Note that Pandas has a lot of functionality, and we cannot cover it all in this lecture. This lecture should provide you with a good foundation to learn more on your when the time comes. 

If you need to look at the detail of a specific function, visit the [Pandas documentation](https://pandas.pydata.org/docs/).

## Importing `pandas`
Below we import `pandas` using its common alias `pd`.

In [ ]:
import pandas as pd

## Series
`pandas` has a data type called series. A series is a single column of data. Series, however, allow for custom indices. For instance, below we define a series that contains the winning NBA team for each year from 2010 to 2022.

In [ ]:
team_names = ["Lakers", "Mavericks", "Heat", "Heat", "Spurs", "Warriors", "Cavaliers",
         "Warriors", "Warriors", "Raptors", "Lakers", "Bucks", "Warriors"]
years = list(range(2010,2023))

nba = pd.Series(data = team_names, index = years, name = "NBA Finals Champions")
print(nba)

### Series Indexing
Indexing works much like lists except for now, we have to use the indicies we provided to `pd.Series`:

In [ ]:
print(f"The winners of the 2012 NBA finals were the {nba[2012]}.")

In [ ]:
print(nba[[2012, 2014, 2016]]) # Print a subseries

Like a dictionary, we can reextract a list of the indices and values using `.index` and `.values` respectively.  

In [ ]:
print(nba.index)
print(nba.values)

### Series Methods
Series objects have methods, some of which we demonstrate below.

In [ ]:
print(nba.head(3)) # prints first three entries

In [ ]:
print(nba.tail())  # prints last five entries by default

In [ ]:
nba.unique() # returns a list of all teams that won at least once

### Series Plots
We can also easily plot series using methods! Below, we define a series of containing the annual US unemployment rate (example from [QuantEcon](https://datascience.quantecon.org/pandas/intro.html))

In [ ]:
values = [5.6, 5.3, 4.3, 4.2, 5.8, 5.3, 4.6, 7.8, 9.1, 8., 5.7]
years = list(range(1995, 2017, 2))
unemp = pd.Series(data=values, index=years, name="Unemployment")
unemp.plot()

## DataFrames
A DataFrame is a table of data -- you can think of a DataFrame as many series stacked side-by-side. This data structure is very similar to the tabular data you might see in an Excel sheet. 

Like series, DataFrames have row indices, but they also have column labels or names. Having two set of indices allows us to refer to individual columns or rows when needed. 

Below, we define a DataFrame of unemployment rates for various regions in the US (example from [QuantEcon](https://datascience.quantecon.org/pandas/intro.html))

In [ ]:
data = {
    "NorthEast": [5.9,  5.6,  4.4,  3.8,  5.8,  4.9,  4.3,  7.1,  8.3,  7.9,  5.7],
    "MidWest": [4.5,  4.3,  3.6,  4. ,  5.7,  5.7,  4.9,  8.1,  8.7,  7.4,  5.1],
    "South": [5.3,  5.2,  4.2,  4. ,  5.7,  5.2,  4.3,  7.6,  9.1,  7.4,  5.5],
    "West": [6.6, 6., 5.2, 4.6, 6.5, 5.5, 4.5, 8.6, 10.7, 8.5, 6.1],
    "National": [5.6, 5.3, 4.3, 4.2, 5.8, 5.3, 4.6, 7.8, 9.1, 8., 5.7]
}
years = list(range(1995, 2017, 2))

unemp_region = pd.DataFrame(data, index=years)
unemp_region

### DataFrame Indexing
Indexing dataframes requires use of `.loc` as shown below

In [ ]:
ne_1995 = unemp_region.loc[1995, "NorthEast"]
print(f"The 1995 Unemployment Rate for the North East Region is {ne_1995}%")

In [ ]:
unemp_region.loc[[1995, 2005], "South"] # Multiple at once

In [ ]:
unemp_region["MidWest"] # extracts whole column, doesn't need to use .loc

### Column Computations
We can do computations with columns of a DataFrame. First let's see what type a column of a DataFrame has and try some computations out.

In [ ]:
type(unemp_region["MidWest"])

In [ ]:
print(unemp_region["West"] / 100) # turn percentage to proportion

In [ ]:
print(unemp_region["West"] - unemp_region["MidWest"]) # can subtract columns

### Data Types
We've seen that Series and DataFrames can contain numbers or strings. Using `.dtype` on a series of `.dtypes` on a DataFrames tells us the datatype of each column. DataFrames have a few types:
- Booleans
- Floating point numbers
- Integers
- Dates
- Categorical Data
- Everything else (objects)

In [ ]:
print(unemp.dtype)
print(nba.dtype) # strings are generic objects in Pandas

In [ ]:
print(unemp_region.dtypes)

### Creating New Columns
If `df` is a dataframe, we can create a new column by simply writing
`df["New Column Name"] = new_values` where `new_values` has the same number of elements as `df` has rows. Below, we create an average of all region's unemployment rates.

In [ ]:
unemp_region["UnweightedMean"] = (unemp_region["NorthEast"] +
                                  unemp_region["MidWest"] +
                                  unemp_region["South"] +
                                  unemp_region["West"])/4
unemp_region.head()

### Changing Individual Values
It can be done using `.loc`

In [ ]:
unemp_region.loc[1995, "UnweightedMean"] = 0.0
unemp_region.head()

## Renaming Columns
To rename columns, we first must define a dictionary where the old column names we want to replace are the keys of the dictionary and the new column names are the values. Then, we use the DataFrame method `.rename()`. `.rename` does not replace the column names, it creates a copy of the dataframe with the new column names, so you need to assign your dataframe to the output of `.rename()` or use the option `inplace = True`.

Below, we rename all region columns to abbreviated names. 

In [ ]:
names = {"NorthEast": "NE",
         "MidWest": "MW",
         "South": "S",
         "West": "W"}
unemp_region.rename(columns=names)
print(unemp_region.head())

In [ ]:
unemp_region  = unemp_region.rename(columns=names)
print(unemp_region.head())

## Loading Data
It would be wildly inefficient if we had to define DataFrames manually, especially when dealing with massive datasets. Luckily, we can load data from our computers or from a URL using the `pd.read_csv()`. below, we load monthly state-level unemployment data from a URL. If we were loading fom our computer, we would replace the url with a local file path.

In [ ]:
url = "https://datascience.quantecon.org/assets/data/state_unemployment.csv"
unemp_raw = pd.read_csv(url, parse_dates=["Date"])
unemp_raw.head(3)

`read_csv` will automatically try and determine what datatype each column is. With this particular dataset, Pandas needs a little help, so we let it know the column "Date" should be interpreted as a date data type. 

## Quick Data Manipulation
We are going to quickly manipulate the data we loaded, so we can illustrate more examples. Do not worry about understanding how the code below works. We will discuss that more later. Let's just talk about how the data looks after we do it. 

In [ ]:
unemp_all = (
    unemp_raw
    .reset_index()
    .pivot_table(index="Date", columns="state", values="UnemploymentRate")
)

states = [
    "Arizona", "California", "Florida", "Illinois",
    "Michigan", "New York", "Texas", "Colorado"
]
unemp = unemp_all[states]

unemp.head()

## What Changed?
We've manipulated the data so every row is a month (month is now the row index) and every column is a state. Each entry is the unemployment rate of that state in the given month.  Since every entry is a number, we can use the plot method on the whole DataFrame!

In [ ]:
unemp.plot(figsize=(8, 6))

## New Indices
Now our data is indexed by dates! Let's try and access all unemployment rates on 2015-01-01.

In [ ]:
print(unemp.loc["01/01/2015", :])
unemp.loc["01/01/2015":"03/01/2015", :]# we can also view many dates at once.

## DataFrame Aggregations
Simply put, aggregation is simply an operation that combines multiple values into a single value. One example of this is an average -- an average takes a collection of numbers (e.g. `[1, 2, 4, 5]`) and returns its average (e.g. 3). 

Pandas has many built-in aggregations such as:
- Mean/Average (`mean`)
- Mode (`mode`)
- Median (`median`)
- Maximum (`max`)

among others. 

For instance, we can see what Colorado's median and maximum unemployment rate was. 

In [ ]:
co_unemp = unemp_raw.loc[unemp_raw['state'] == "Colorado"]

print(co_unemp["UnemploymentRate"].median())
print(co_unemp.UnemploymentRate.max())

### Custom Aggregations
Using a custom function that takes a series as an input and the `.agg()` method, we can create custom aggregations. Below, we creat a function that classifies a state as high or low unemployment based on its average unemployment. 

In [ ]:
def high_or_low(s):
    """
    This function takes a pandas Series object and returns high
    if the mean is above 6.5 and low if the mean is below 6.5
    """
    if s.mean() < 6.5:
        out = "Low"
    else:
        out = "High"

    return out

In [ ]:
print(unemp.agg(high_or_low)) 

In [ ]:
unemp.agg(['min', 'max', high_or_low]) # Can do multiple aggregate at once -- returns DataFrame not series

## Transformations 
Sometimes we want to transform the columns according to a function that also return a series. Some built-in transformations include
- Cumulative sum/max/min/product (`cum(sum)`, `cum(max)`, etc.)
- Differences (`diff`)
- Elementwise addition/subtraction/multiplication/divison (`+`, `-`, `*`, `/`)
- Percent change (`pct_change`)
- Number of occurence for each distinct value (`value_counts`)
- Absolute value (`abs`)

For instance, we could calculate the change in unemployment rate from one month to the next for each state.

In [ ]:
unemp.head()

In [ ]:
unemp.diff().head()

## Custom Series Transforms with `.apply()`
If we want to transform each column as we did above, but with a custom function, we can do this using the `.apply()` method. 

To do this, we need to write a function that takes in a series and returns a series. Then we pass the function to the `.apply()` method. Below, we define a function that standardizes each value in a column. 

In [ ]:
def standardize_data(x):
    """
    Changes the data in a Series to become mean 0 with standard deviation 1
    """
    mu = x.mean()
    std = x.std()

    return (x - mu)/std

In [ ]:
std_unemp = unemp.apply(standardize_data)
std_unemp.head()

## Index Revisited
Indices were introduced as being labels for rows, but they also serve other functions. Namely, a given data point will be associated with a specific row index until we as the users explicitly break that association. 

Let's discuss the code below. What do you think will happen when we add `a + b`?

In [ ]:
a = pd.Series({1:5, 2:5, 3:20, 4:14, 10:5})
b = pd.Series({1:5, 4:10, 3:10, 6:2})
print(a)
print(b)


In [ ]:
print(a+b)

`pandas` knows that when we add two Series together, we want elements with the same index to be added! If a given index does not exist in both Series, it returns `NaN`. This means we don't have to concern ourselves with whether the data was shuffled around accidentally when performing operations on DataFrames and Series. 

## Saving Files Formats
For this class, we will only use CSVs. Nevertheless, it is important to understand the various file formats that exist. We will review some file formats discussed on [QuantEcon](https://datascience.quantecon.org/pandas/storage_formats.html).

## Writing to CSVs
Pandas DataFrames have a method called `.to_csv()` that requires a file path for the file. Generally, the file path is relative to the `working directory`.
You can think of a working directory as the default directory that your Jupyter notebook operates in. 

You can check your present working directory by using `pwd()`


In Jupyter Open, it will save onto the cloud, but you can download it to you local computer easily. 


Below, we save our standardized unemployment dataframe to a csv with the title "stand_unemp.csv" in the folder `lecture_generated_objects`.

In [ ]:
%pwd

In [ ]:
std_unemp.to_csv("stand_unemp.csv")

## Cleaning Data
After you obtain some data and load it into Python for the first time, it is rarely ready for analysis. Not only do you have to familiarize yourself with the data, what variable it contains, how those variables are coded, etc., but also you also may need to clean the data. **Data cleaning**  refers to basically anything you do after you have loaded the data into Python but before you analyze it. This includes but is not limited to:
- correcting typos or incorrectly coded data points
- data may not be of the right type
- throwing out or filling in missing data
- joining datasets across many files into one DataFrame
- removing outliers
- reshaping data into the needed form
- taking a subsample of the data


In that sense, you could say the remainder of what we learn about Pandas is about data cleaning. Whatever you call it, `pandas` includes tools to deal with all of these issues data presents us. Consider the following mock dataset.

In [ ]:
import math
df = pd.DataFrame({"numbers": ["#23", "#24", "#18", "#14", "#12", "#10", "#35"],
                   "nums": ["23", "24", "18", "14", math.nan, "XYZ", "35"],
                   "colors": ["green", "red", "yellow", "orange", "purple", "blue", "pink"],
                   "other_column": [0, 1, 0, 2, 1, 0, 2]})
df

## Type Issues
When working with data, the columns may not be of the `dtype` we want. This can stop us from performing certain operations on the data. Below, we try and use the aggregator method `.mean()` on the column `numbers`. 

In [ ]:
# Uncomment this line and see the error
df.numbers.mean()

As the error tells us, the numbers column is actually a string, so we need to convert the numbers column to a numeric data type. First, however, we must get rid of the # symbol.

We could do this by looping through each element of the column and use the string method `.replace()`, but that is highly inefficient. Instead, we can apply string methods (not just `.replace()`) directly to pandas columns that contain strings to create a new column!

To make it so our new column is a numeric `dtype`, we simply use the pandas function `pd.to_numeric()`. Alternatively, we can use the Series method `astype()` which take a Python type as an argument and tries to convert the column to that type.

In [ ]:
df["numbers"].str.replace("#", "")

In [ ]:
# replace the pound symbol
df["numbers_str_1"] = pd.to_numeric(df["numbers"].str.replace("#", ""))
print(df["numbers_str_1"].mean())

df["numbers_str_2"] = (df["numbers"].str.replace("#", "")).astype(float)
print(df["numbers_str_2"].mean())

## Missing Data
**Missing data** is an important consideration when performing an analysis. In its most literal form, missing data is best thought of in the context of tables. A table has missing data when one or more cells in that table has no value in it or a value that is interpreted as missing. In pandas, this missing data is represented by the `NaN` (not a number) symbol. 

Below, we illustrate some strange behavior that NaNs exhibit and the DataFrame method `is.null()` which can help you detect and consequently, deal with missing values.

In [ ]:
# NaNs have some weird behaviors that require specific functions
print(math.nan == math.nan) # this won't work for detecting nan values
math.isnan(math.nan) # have to use this

In [ ]:
print(df.isnull()) # see which cells are missing

In [ ]:
print(df.isnull().any()) # see if there is a single missing cell

We can also detect if individual rows or columns have missing data.

In [ ]:
print(df.isnull().any(axis=0))

In [ ]:
print(df.isnull().any(axis=1))

## What can we do with missing data?
- Throw out missing data (`.dropna` method)
- Fill in the missing data (`.fillna` method)
- Model missingness (complicated and beyond the scope of this course) 

Let's see what `.dropna()` and `.fillna()` do and discuss.

In [ ]:
print(df.dropna())

In [ ]:
print(df.fillna(32))

## "Tidy" Data
- Concept to help us understand how we should reshape our data
- The concept of tidy data is well-summarized by the following quote from Hadley Wickham's "[Tidy Data](https://www.jstatsoft.org/index.php/jss/article/view/v059i10/v59i10.pdf)":
>A dataset is messy or tidy depending on how rows, columns and tables are matched with observations, variables, and types. In tidy data: 
>1. Each variable forms a column.
>2. Each observation forms a row.
>3. Each type of observational unit forms a table
- This view usually leads to the question, what is an observation in this context? (e.g year, country, individual, firm, year-country etc.)
- This observation-level identifier (e.g. 1993, France, John Smith, Amazon, France-1993, etc.) can serve as your `DataFrame` index as it should be unique.

## Reshaping Data
Sometimes, data is not in the shape you need it to be in for analysis. To illustrate this, we consider a dataset on a few basketball players and their points scored over various seasons. What is the unit of observation in the `bball` DataFrame? 

In [ ]:
url = "https://datascience.quantecon.org/assets/data/bball.csv"
bball = pd.read_csv(url)

bball

## Long vs. Wide Tabular Data
We can make this table longer (more rows and fewer columns) or wider (fewer rows and more columns) depending on our needs. 

### Wide to Long
If we want to make data longer, there are three potential functions/methods you can use: `.melt()`, `.stack()`, and `wide_to_long()`. 

### Long to Wide
When we want to make our data wider, we can use `.unstack()`,`.pivot()`, and `.pivot_table()`. 

## Wide to Long with `.melt()`
Below, we use `.melt()` to go from wide to long. The areguments we feed to `id_vars` are the variables we want to keep as variables. After observing what this function, what is an observation now? What did `.melt()` do?

In [ ]:
bball_long = bball.melt(id_vars=["Year", "Player", "Team", "TeamName"])
bball_long

## Long to Wide with `.pivot()` and `.pivot_table()`
Related to pivot tables you may have seen in excel. Both help us go from long to wide. `.pivot_table()` is a generalization of `.pivot()`, so we will focus on it. 

### `pivot_table()`
- Choose columns the index
- Choose columns as variables
- One column serves as the values
- Duplicates are aggregated -- can specify how

Easiest to understand with an example. Think of whch of these examples will need some form of aggregation and which will not. 

In [ ]:
# Here we keep the team and year as our index and move the players to the columns. 
bball.pivot_table(index=["Year", "Team"], columns="Player", values="Pts")

In [ ]:
# Alternatively, we can put the team names as the columns with players
bball.pivot_table(index="Year", columns=["Player", "Team"], values="Pts")

In [ ]:
# This requires aggregation. Why? What is used by default?
bball_pivoted = bball.pivot_table(index="Year", columns="Player", values="Pts")
bball_pivoted

In [ ]:
# Can change aggregation type
bball.pivot_table(index="Year", columns="Player", values="Pts", aggfunc=max)


In [ ]:
# We can view how many duplicates each year-player combinations has
bball.pivot_table(index="Year", columns="Player", values="Pts", aggfunc=len)


We can actually take our long dataframe earlier and use `.pivot_table()` to make it wider.

In [ ]:
bball_wide = bball_long.pivot_table(
    index="Year",
    columns=["Player", "variable", "Team"],
    values="value"
)
bball_wide

In [ ]:
bball

### Aside: Indexing Revisited
Before proceeding with `stack()` and `unstack()`, we first must learn three more methods, `.set_index()`, `.reset_index()`, and `.T` (transpose). First, we notice that a player-year uniquely determine a row, so we can set the index as a player-year

In [ ]:
bball2 = bball.set_index(["Player", "Year", "Team"])
bball2

In [ ]:
# Transpose switches columns and rows
bball3 = bball2.T
bball3.head()

In [ ]:
# Demonstrate .reset_index()
bball3.reset_index() # Previous index gets saved as a column call index.

## Wide to Long with `.stack()`
Below, we demonstrate some uses of `.stack()` on `bball_wide`. In the example below, notice that `.stack()` moves the top column by detault. 

In [ ]:
# before
bball_wide

In [ ]:
# after
bball_wide.stack()

In [ ]:
# This allows us to easily get player level means for each stastic
player_stats = bball_wide.stack().mean()
player_stats

### Choose your Level
What if, however, we want to move the players into the index instead of the team? To do this, we simply specify which level we want to move using the level argument. This allows us to easily get team averages.

In [ ]:
bball_wide.stack(level="Player")


In [ ]:
bball_wide.stack(level="Player").mean()


### Multiple Levels
We can also stack multiple levels simultaneously by specifying many levels.

In [ ]:
bball_wide.stack(level=["Player", "Team"])


## Long to Wide with `.unstack()`
Recall the `player_stats` series we defined earlier. Using `.unstack()`, we can turn this series back into a DataFrame. This form makes it easy to make a bar plot by player or by statistic.

In [ ]:
player_stats.unstack()

In [ ]:
player_stats.unstack().plot.bar()
player_stats.unstack(level = "Player").plot.bar()


## Merging Data
Frequently, we want to perform analysis on data that comes from two or more different sources or are spread across many files. For example, we may be interested in the prices of a company's stock and its earnings but our data might be spread out across many files.
- Different files for different years
- Different files for different companies
- Different files for stock prices and earnings

Depending on our unit of observation, these different situations might require different ways of **merging data** -- that is getting all of the data into a single DataFrame.

The three main ways to merge data in pandas are:
1. `pd.concat([df1, df2,...])`
2. `pd.merge(df1, df2)`
3. `df1.join(df2)`

In [ ]:
### Load Datasets Used for Illustration
# from World Development Indicators (WDI). Units trillions of 2010 USD
url = "https://datascience.quantecon.org/assets/data/wdi_data.csv"
wdi = pd.read_csv(url).set_index(["country", "year"])
wdi.info()

wdi2017 = wdi.xs(2017, level="year")


# Data from https://www.nationmaster.com/country-info/stats/Geography/Land-area/Square-miles
# units -- millions of square miles
sq_miles = pd.Series({
   "United States": 3.8,
   "Canada": 3.8,
   "Germany": 0.137,
   "China": 3.7,
   "Brazil": 3.3
}, name="sq_miles").to_frame()
sq_miles.index.name = "country"



In [ ]:
# View datasets so we can compare later
wdi2017

In [ ]:
# View datasets so we can compare later
sq_miles

### `pd.concat()`
"concat" stands for concatenation. Simply put, concatenating DataFrames entails stacking them on top of eachother or side-by-side. This,however, is done intelligently though as indices must match. The argument `axis` allows user to specify which way Python should stack the DataFrames. 

We show two concatenations below. Which one looks more "tidy?"

In [ ]:
# The two dataframes have no columns in behavipr
pd.concat([wdi2017, sq_miles], axis = 0)

In [ ]:
# China and Brazil are missing from wdi2017
pd.concat([wdi2017, sq_miles], axis = 1)

In [ ]:
# Can obtain per square mile statistics now! Remember Exports is in trillions and sq_miles is in millions
temp = pd.concat([wdi2017, sq_miles], axis=1)
temp["Exports"] *(1_000_000) / (temp["sq_miles"] )

### `pd.merge()` or `df.merge()`
Brings columns from one DataFrame into another, but instead of just combining them using an index, it uses one or more keys (variables found in both datasets) to combine the data.

### `df.join()`
This is a less general version of merge that is a DataFrames method instead of a function.

Merging & joining are not straightforward concepts, so let's start with some examples. 

In [ ]:
pd.merge(wdi2017, sq_miles, on="country")

### What do we see?
- China, Brazil, and the UK were excluded. What did those countries have in common (with regards to the data)?
- Besides that, similar to `pd.concat([wdi2017, sq_miles], axis = 1)`

The power of merging is more obvious when we merge on a variable that is not unique.

In [ ]:
# load wdi data for 2016 and 2017
wdi2016_17 = wdi.loc[pd.IndexSlice[:, [2016, 2017]],: ]
wdi2016_17

In [ ]:
# merge `wdi2016_17` data with `sq_miles`
pd.merge(wdi2016_17, sq_miles, on="country")

### What do we see?
- China, Brazil, and the UK were excluded again.
- We lost the `year` variable. 
- Each country has the `sq_miles`value repeated twice -- once for 2016 and once for 2017

To recover the year, simply reset the index, so that it becomes a column instead. If you want, you can set the index back again using `.set_index()`.

In [ ]:
temp = pd.merge(wdi2016_17.reset_index(), sq_miles, on="country").set_index(["country", "year"])
temp

In [ ]:
print(1_000_000 * temp["Consumption"] / temp["sq_miles"])

## Multiple Column Merges
What if we have multiple datasets with country-year observations that we want to merge together? Luckily, `pd.merge()` makes this easy. Below, we load another datset that contains the 2000-2017 populations of the folowing countries:
- Canada
- Germany
- United States
- United Kingdom

Then, we merge this new dataframe with `wdi2016_17`

In [ ]:
## Loading a dataset on population
# from WDI. Units millions of people
pop_url = "https://datascience.quantecon.org/assets/data/wdi_population.csv"
pop = pd.read_csv(pop_url).set_index(["country", "year"])
pop

In [ ]:
wdi_pop = pd.merge(wdi2016_17, pop, on=["country", "year"])
wdi_pop

### What do we see?
- China, Brazil, and the UK were excluded again.
- We didn't lose the the `year` variable even though it was the index because we it was a variable we merged on! 
- Population matches as we would expect.


Now we can use this to dataset to get per-capita measurements!

In [ ]:
1_000_000 * wdi_pop["GDP"] / wdi_pop["Population"]

## Other `merge` Arguments
`pd.merge()` has optional arguments we have not discussed. To use these, you should understand the concept of `left` and `right`. Namely, the DataFrame that serves as the first argument in `merge()` is the `left` DataFrame. The second DataFrame argument is the `right` DataFrame. 

**Example:** `pd.merge(df_left, df_right, on = variable)`.


### `on`, 
This is technically an optional argument. If it is not specified, all column names (**not indices!**) that appear in both `left` and `right` DataFrames will be used. 


In [ ]:
# This will not work if we do not .reset_index() 
pd.merge(wdi2016_17.reset_index(), pop.reset_index()).set_index(["country", "year"])


### `left_on` and `right_on`
Use this when the columns you want to match on are called something different.

In [ ]:
# define DataFraem with different index name
cont_df = pd.Series({"Canada":"North America", 
                  "Germany":"Europe",
                  "China":"Asia",
                  "Brazil":"South America",
                  "Kenya":"Africa"}).to_frame()
cont_df.index.name = "country or something else"
cont_df = cont_df.rename(columns = {0:"Continent"})
cont_df

In [ ]:
# Reset index on the variable we want to keep as they will be identical
pd.merge(sq_miles.reset_index(), cont_df, left_on = "country", right_on = "country or something else")

### `left_index` and `right_index`
If we want to merge on the index on our `left` or `right` DataFrames, we can set  `left_index` or  `right_index` equal to `True` respectively. 

In [ ]:
pd.merge(sq_miles.reset_index(), cont_df, left_on = "country", right_index = True)

In [ ]:
pd.merge(wdi2017, cont_df, left_index = True, right_index = True)

### `how` 
This far, only keys (countries and years) that have been in both the `left` and `right` dataset appear in the merged dataset. This behavior is caused by the default value of `how`. What if we want these keys to stick around even if they don't exist in the other dataset?

This is one of the two most difficult Pandas concepts we will cover and requires keeping track of which DataFrame is `left` and which one is `right`.  

`how` can take four arguments:
- "left"
- "right"
- "inner"
- "outer"

Below, is a helpful image from [QuantEcon](https://datascience.quantecon.org/pandas/merge.html) that shows what each argument does. Can you guess which is the default one?

<img src="https://datascience.quantecon.org/_images/merge_venns.png" alt="Alternative text" />


### Argument descriptions
- "left" uses all keys from `left`, regardless of their presence in `right`
- "right" uses all keys from `right`, regardless of their presence in `left`
- "inner" uses only keys that appear in both `left` and `right`
- "outer" uses all keys from both `left` and `right`

Let's see some example to solidify our understanding. 

In [ ]:
# left
pd.merge(cont_df, wdi2017, left_index = True, right_index = True, how = "left")

In [ ]:
# right
pd.merge(cont_df, wdi2017, left_index = True, right_index = True, how = "right")

In [ ]:
# inner
pd.merge(cont_df, wdi2017, left_index = True, right_index = True, how = "inner")

In [ ]:
# outer
pd.merge(cont_df, wdi2017, left_index = True, right_index = True, how = "outer")

## Aggregation with GroupBy
Frequently, we want to get aggregate statistics (likes averages, modes, and medians) on a group level. For instance, we might want ot use data on individuals' heights and ages to see how average height varies by age. 

`.groupby()` is an essential tool for such aggregation and is probably the most difficult Pandas concept we will learn. Using `.groupby()`, we can divide our data into groups and use aggregators to get statistics on the group-level.

## Three Steps
This process can broken down into three steps:
- **Split** the data into groups. These groups will be determined by the values of one or more columns.
- For each group, **apply** the same function or process (e.g. mean, median, etc.)
- Take the output of that function or process for each group and **combine** them into a single DataFrame where group identifiers serve as the index. 

To begin, we will demonstrate `.groupby()` with a toy example

In [ ]:

df = pd.DataFrame({
    "A" : [1, 1, 1, 2, 2, 2, 2, 1],
    "B" : ["Green", "Red", "Green", "Green", "Blue", "Red", "Green", "Red"],
    "C":  [1.0, 2.0, 3.0, math.nan, 5, math.nan, 2.2, 3.4],
    "D" :  [1, 3, 5, 7, 5, 3, 1, 3]
})
df

In [ ]:
# First group by a
gbA = df.groupby("A")
type(gbA)

### `.get_group()`
Note the type is a DataFrameGroupBy. This type has a method `.get_group()` that allows us to view keys by supplying it the key of the group we want to see. See if you can guess what the code will display before we run it.

In [ ]:
(gbA.get_group(1))

In [ ]:
(gbA.get_group(2))

### Multiple Column GroupBy
Much like we can merge two DataFrames on multiple columns, we can group a DataFrame by multiple columns.

In [ ]:
gbAB = df.groupby(["A", "B"])

In [ ]:
# We can check which groups exist and which indices can be found in each group
gbAB.groups

In [ ]:
# Indices are now tuples
gbAB.get_group((1, "Green"))

In [ ]:
gbAB.get_group((2, "Blue"))

## Aggregators
Once we have a GroupBy object, we have completed the first step of splitting the data into two groups. We can then apply aggregator methods to our data set which simultaneously applies the functions to each groups and combines the results into a DataFrame. 

Looking at the below, how did the `.sum()` aggregator deal with the `NaN` values? 
How did the `.count()` function deal with with NaNs values?

In [ ]:
# Print original df for comparison
df

In [ ]:
# Sum all columns (not indices!)
gbAB.sum()

In [ ]:
# Now we how many observations or rows are in each group
gbAB.count()

### Custom Aggregators for GroupBy
Custom Aggregators also work on GroupBy objects. Simply define a function that takes a DataFrame or Series as an input and returns a Series or a single value. As before, we feed this function to the `.agg()` method.  Below, we define a function that returns the number of missing data points in each column. 

In [ ]:
# Define function that takes Data Frame
def num_missing(df):
    return df.isnull().sum()

In [ ]:
# .agg(num_missing) applies num_missing to each group individually then combines the results
gbAB.agg(num_missing)


### Custom Transforms for GroupBy
We can also apply custom transforms to GroupBy objects by using the `.apply()` method

In [ ]:
# Define function that returns rows where column B is equal to "Green" of a Data Frame
def find_green_rows(df):
    return df.loc[df["B"] == "Green", :]

In [ ]:
gbAB.apply(find_green_rows)


In [ ]:
# Note the index changed -- what happened? 
gbAB.apply(find_green_rows).index

## Working with Date & Time Data
Sometimes, we will have a date or date and time data that we want to work with. Sometimes, that entails extracting some component of the date (e.g. month or day of the week) to save as its own variable. Other times, our dates may be stored as strings or a series of numbers, but we want them to be handled as dates (e.g. what date is one day from a given date, was the day before a Monday, how much time elapsed between two datetimes, etc.)

`datetime` objects and their associated methods help us deal with all of these scenarios and more!

### Parsing Dates from Strings
First, we demonstrate how to convert a string containing a date into a `datetime` object. Like other type conversions, we simply use `pd.to_datetime`. It is important that the string is formatted a certain way, so that its contents can be interpreted as a `datetime`.

After running the cell below, observe what happens. What happens when we do not provide a time?

In [ ]:
# Convert date string to datetime
christmas_str = "2017-12-25"
christmas_date = pd.to_datetime(christmas_str)
christmas_date

In [ ]:
# Convert date and time string to datetime
christmas_time_str = "2017-12-25, 13:34:34"
christmas_time = pd.to_datetime(christmas_time_str)
christmas_time

In [ ]:
# Convert list of dates
kwanza_dates = pd.to_datetime(["2023-12-26", "2023-12-27", "2023-12-28", "2023-12-29", "2023-12-30", "2023-12-31", "2024-01-01"])
kwanza_dates

### Flexible Date Formats
As you can see in the cell directly below, `pd.to_datetime` can parse many date formats. Uncomment the cell below that one, however, and you will see it cannot handle everything.

In [ ]:
for date in ["December 25, 2017", "Dec. 25, 2017",
             "Monday, Dec. 25, 2017", "25 Dec. 2017", "25th Dec. 2017"]:
    print("pandas interprets {} as {}".format(date, pd.to_datetime(date)))

In [ ]:
# We create a very weird date time string
weird_time = "Second:45, Minute:32, Hour:21, Day:3, Month:3, Year:2032"

In [ ]:
# uncomment below and see it fail
#pd.to_datetime(weird_time)

### Specify the Format for String Conversion
Luckily, we can still deal with dates that are formatted like `weird_time` by communicating the pattern to `pd.to_datetime()` using another string. 

Below, we define such a string and feed it to the argument `format`. Can you figure out what the % symbols follows by a letter below signify?

In [ ]:
weird_pattern_str = "Second:%S, Minute:%M, Hour:%H, Day:%d, Month:%m, Year:%Y"
pd.to_datetime(weird_time, format = weird_pattern_str)


### Format `datetime` as string
We may also want to format `datetime` as a string or extract a certain value from the `datetime` and turn it into a number. We can t dates as strings using the `.strftime()` method.

Run the cell below and observe the output. What do you thing `%B` does? How about `%e`? 

[here](https://strftime.org/) is a usefule reference for some of the possible `.strftime()` formatting options.


In [ ]:
string = "Kwanza starts on " + kwanza_dates[0].strftime("%B %d") + "th and ends on " + kwanza_dates[len(kwanza_dates)-1].strftime("%B%e") + "st."
string

### Date Properties
Sometimes, we may have `datetime` variables and we want to get numeric values (e.g. the hour, day of the month, year, etc.). Below, we demonstrate how to access some of these values.

In [ ]:
print(pd.to_datetime("2012-11-15").year)
print(pd.to_datetime("2023-07-25").dayofweek)
print(pd.to_datetime("2012-11-15").day)

## Dates as Indices
In a previous example, we have observations that were country-year and country-year pairs defined our observations uniquely. Using a `DateTimeIndex` instead of a numeric index give us access to many powerful tools. Below, we load data on daily BitCoin to US dollar (USD) exchange rates from 2014 onwards. 

Note how our index is a `DateTimeIndex`! 

In [ ]:
# %pip install quandl for installing quandl

In [ ]:
btc_usd = quandl.get("BCHARTS/BITSTAMPUSD", start_date = pd.to_datetime("01-01-2014"))
btc_usd.info()

In [ ]:
btc_usd.head(3)

### Flexibilie Indexing Syntax
Normally, we can reference a row only by specifically referencing its index. If we want to select many indices, we can use slices or other collections to select them. 

With dates, however, we have a different way to select many rows at once that leverages the same flexibility of `pd.to_datetime()`. For example, below we select all prices in 2019 with a single command.

In [ ]:
btc_usd.loc["2019"] # Select BTC prices in 2019

In [ ]:
# select end of month price data 
btc_usd.loc[btc_usd.index.is_month_end]

In [ ]:
# Select all prices in October 2018
btc_usd.loc["October 2018"]

In [ ]:
# Can specify a range od ates
btc_usd.loc["October 2018":" December 2019"]

### Date Index Properties
Earlier, we demonstrated how to extract individual numerical elements from a date (year, day of the week, etc.) We can do the same thing with a `DateTimeIndex` object.  These operations return an Int64Index object. If we are operating on a column of `DateTime` objects, then we have to add an extra `.dt` betweent the object and the commands.

In [ ]:
# Index example
print(type(btc_usd.index.year))
btc_usd.index.year

In [ ]:
btc_date_column = btc_usd.reset_index()
btc_date_column["Date"].dt.year

## Leads and Lags Using `df.shift()`
When dealing with what's known as time series data, we sometimes want to compare data from one datetime with another datetime. For example, we might be interested in calculating Bitcoin's price change from one day to the next. To do this, we need to associate prices from one day (e.g May 1st, 2015) with prices from the next day (e.g. May 2nd, 2015). 

We can do this easily using `df.shift(n)` where `n=1`. Think about why the first rows is all `NaN`.

In [ ]:
# Before df.shift()
btc_usd

In [ ]:
# after df.shift()
btc_usd.shift()

Now, we can take advantage of Python's indexing to calculate the percentange change in all of the column variables from one day to the next! 

In [ ]:
((btc_usd - btc_usd.shift()) / btc_usd.shift()).head(3)

### Shifting by Different Amounts
If we wanted two day price changes we can chose `n=2`. First, let's see what shifting by 2 does. Why are the top two columns now all `NaN`?

In [ ]:
btc_usd.shift(2).head()


So far positive numbers have been giving us what are called **lags**. That is, we have been creating associations between a date and data from an **earlier** date. **Leads** are the opposite. In this case, we associate a date with data from a **later** date. To get leads, we use negative numbers.

Below, we print the last three rows of `btc_usd.shift(-2)`. What do you expect to see?

In [ ]:
btc_usd.shift(-2).tail(3)